In [4]:
from contextlib import asynccontextmanager
import asyncio

/usr/lib/python3.11/enum.py:714: RuntimeWarning: coroutine 'main' was never awaited
  return cls.__new__(cls, value)


In [3]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
@asynccontextmanager
async def make_connection(name):
    print(f"Connecting... {name}")
    yield name
    print(f"Connected! {name}")

async def main():
    async with make_connection("A") as a:
        print(f"Using connection: {a}")

asyncio.run(main())

Connecting... A
Using connection: A
Connected! A


In [6]:
import asyncio
from contextlib import AsyncExitStack

async def get_connection(name):
    class Ctx():
        async def __aenter__(self):
            print(f"ENTER... {name}")
            return name
        async def __aexit__(self, exc_type, exc, tb):
            print(f"EXIT! {name}")
    return Ctx()

async def main():
    async with await get_connection("A") as a:
        async with await get_connection("B") as b:
            print(f"Using connections: {a} and {b}")



asyncio.run(main())

ENTER... A
ENTER... B
Using connections: A and B
EXIT! B
EXIT! A


In [ ]:
async def main():
  async with AsyncExitStack() as stack:
      a = await stack.enter_async_context(await get_connection("A"))
      b = await stack.enter_async_context(await get_connection("B"))
      print(f"Using: {a} and {b}")
  # On exit: first b.__aexit__, then a.__aexit__

asyncio.run(main())




ENTER... A
ENTER... B
Using: A and B
EXIT! B
EXIT! A


In [7]:
import asyncio
from contextlib import AsyncExitStack

async def get_connection(name):
    class Ctx():
        async def __aenter__(self):
            print(f"ENTER... {name}")
            return name
        async def __aexit__(self, exc_type, exc, tb):
            print(f"EXIT! {name}")
    return Ctx()

# async def main():
#     async with await get_connection("A") as a:
#         async with await get_connection("B") as b:
#             print(f"Using connections: {a} and {b}")

async def main():
    async with AsyncExitStack() as stack:
        a = await stack.enter_async_context(await get_connection("A"))
        if a == "A":
            b = await stack.enter_async_context(await get_connection("B"))
            print(f"Using connections: {a} and {b}")

        async def customCleanup():
            print("Custom cleanup logic here")

        stack.push_async_callback(customCleanup)
        print(f"Doing work with {a} and maybe {locals().get('b')}")
        await asyncio.sleep(1)

asyncio.run(main())

ENTER... A
ENTER... B
Using connections: A and B
Doing work with A and maybe B
Custom cleanup logic here
EXIT! B
EXIT! A
